# Web Scrapping estatísticas superliga feminina 2018-2019

In [27]:
# Importando os pacotes necessários
from urllib import request
from bs4 import BeautifulSoup
import pandas as pd

### Função Scrapper - Tabela de Jogos

In [28]:
def scrapper_tabela_jogos(url):
    
    # Coletando a página
    with request.urlopen(url) as website:
        page = website.read()

    soup = BeautifulSoup(page, 'html.parser')
    
    # Coletando dados
    rows_data_local = soup.find_all('div', class_="t-col t-col-2 t-col-xs-3 t-col-sm-3 t-hidden-xs t-hidden-sm t-hidden-md Calendar_DIV_Column")
    rows_times = soup.find_all('div', class_="t-col t-col-2 t-col-sm-3 t-col-md-2 t-hidden-xs Calendar_DIV_Column")
    rows_placar = soup.find_all('div', class_="t-col t-col-1 t-col-xs-4 t-col-sm-3 t-col-md-2 Calendar_DIV_Column")
    
    # Coletando os IDs dos jogos
    ids = []

    for row in rows_data_local:
        result = row.find_all('p', class_='Calendar_p_TextRow Calendar_p_TextRow_Italic')
        id_jogo = str(result[0]).split('MatchStatistics.aspx?mID=')[1].split('&')[0]
        ids.append(int(id_jogo))
        
    
    
    # Coletando todos as datas e locais e armazenando em listas
    datas = []
    local = []

    for row in rows_data_local:
        result = row.find_all('span')
        datas.append(result[0].string)
        local.append(result[1].string)

        
    # Coletando os jogos 
    mandante = []
    visitante = []

    for index in range(len(rows_times)):
        time = rows_times[index].find_all('span')
        if index%2 == 0:
            mandante.append(time[0].string)
        else:
            visitante.append(time[0].string)


    # Coletando os placares
    sets_mandante = []
    sets_visitante = []

    for row in rows_placar:
        result = row.find_all("span")
        if len(result) == 3:
            sets_mandante.append(int(result[0].string))
            sets_visitante.append(int(result[2].string))
        else:
            sets_mandante.append(None)
            sets_visitante.append(None)
    
    
    # Gerando Rodada e Turnos
    rodada = []
    turno = []

    t = 1
    rt = 0
    rq = 0
    rs = 0


    for i in range(153):

        if i < 132:
            if i%6==0:
                t = t*(-1)
            if t > 0:
                turno.append('Primeiro Turno')
            else:
                turno.append('Segundo Turno')


            if i%12 == 0:
                rt = rt+1
            rodada.append(rt)

        elif i < 144:
            turno.append('quartas')
            if i%4 == 0:
                rq = rq+1
            rodada.append(rq)

        elif i < 150:
            turno.append('semis')
            if i%2==0:
                rs=rs+1
            rodada.append(rs)

        else:
            turno.append('finais')
            rodada.append(i-149)
        
        
    # Criando DataFrame do Pandas os dados
    df = pd.DataFrame({
        'id_jogo': ids,
        'data_jogo': datas,
        'local_jogo': local,
        'mandante': mandante,
        'sets_mandante': sets_mandante,
        'sets_visitante': sets_visitante,
        'visitante': visitante,
        'rodada': rodada,
        'turno': turno,
    })
    
    df = df.sort_values(by=['turno', 'rodada'])
    return df
    

### Função Scrapper estatísticas

In [191]:
def estatisticas_jogo(id_jogo):
    
    # Coletando dados da página
    url = f"http://cbv-web.dataproject.com/MatchStatistics.aspx?mID={id_jogo}"

    with request.urlopen(url) as website:
        page = website.read()

    soup = BeautifulSoup(page, 'html.parser')
    
    # Coletando as tabelas da página
    tables = soup.find_all('table', class_='rgMasterTable')
    stats_mandante = tables[0]
    stats_visitante = tables[1]
    
    # Coletando os dados
    tabelas = []

    for tabela in [stats_mandante, stats_visitante]:

        # Coletando o body da tabela
        tbody = tabela.find_all('tbody')

        # Coletando as linhas da tabela
        rows = tbody[0].find_all('tr')

        linhas = []
        for row in rows:
            data = row.find_all('span')

            dados = []
            for datum in data:
                dados.append(datum.string)

            linhas.append(dados)

        tabelas.append(linhas)

    # Cabeçalho das tabelas
    th = ['#', 'Jogadora', 'set_1', 'set_2', 'set_3', 'set_4', 'set_5', 'Pontos_Tot', 'BP', 'V-P', 'a', 'Saques_Tot', 'Saques_Err', 'Saques_Ace', 'Rec_Tot', 'Rec_Err', 'Rec_%Pos', 'Rec_%_Exc.', 'b', 'c', 'At_Tot', 'At_Err', 'At_Blk', 'At_Exc.', '% Exc. ', 'd', 'Block_Pts', 'e']
    
    # Removendo a linha de totais
    totais_mandante = tabelas[0].pop()
    totais_visitante = tabelas[1].pop()
    
    totais = [totais_mandante, totais_visitante]
    
    df_mandante = pd.DataFrame(tabelas[0], columns=th).drop(['a', 'b', 'c', 'd', 'e'], axis=1).set_index('#')
    df_visitante = pd.DataFrame(tabelas[1], columns=th).drop(['a', 'b', 'c', 'd', 'e'], axis=1).set_index('#')
    
    return df_mandante, df_visitante, totais

In [194]:
df1, df2, tots = estatisticas_jogo(528)